# A differentiable radio interferometry model for Bayesian MAP estimation

## What do we want
Given a set of visibility $V_{pq}$ observed by a radio interferometer, where $p, q$ are indices for the antennas, we want to solve for the sky brightness (i.e. make an image) $I_{ij}$, where $i, j$ are pixel number.

## The forward model
The foward model takes an image and creates visibility from it. Let's start with the visibility equation/van Cittert-Zernike theorem
$$ V(u, v, w) = \int \int I(l, m) e^{-2\pi i (ul + vm + w\sqrt{1-l^2-m^2})} dl dm,$$

where $l$ is the east-west direction on the sky, m is the north-south component, $n=\sqrt{1-l^2-m^2}$ is the vertical component. $u,v,w$ are the three coordinates for baselines in the interferometer (two antennas make a baseline) in units of wavelengths.

Let's make this easier by assuming a smaller field of view for our image orienting in the up-is-north direction. Then $i=l/\theta, j=m/\theta$, where $\theta$ is the pixel size. Also assume that the interferometer we work with is co-planar with the phase center chosen such that w=0. These assumptions can be loosened without additional inference-time computational cost. Soooo the equation that we will be dealing with is

$$ V_{pq} = \sum_i \sum_j I_{ij} e^{-2\pi i (u_{pq}l(i) + v_{pq}m(j))}, $$
where, I think $u_{pq} = u_p - u_q, v_{pq} = v_p - v_q$, where $u_i,v_i$s are the NS and EW coordinates of each antenna.

Let's stick with the summation implementation for now, because the matrix implementation requires pre-computing a gigantic matrix, which isn't great.

## Bayesian inference.
Now we have the model that gives measurements from the quantity that we are intersted in ($I$). We are interested in optimizing the posterior distribution over the parameters $I$ (the image):
$$p(I | V) \propto prior(I) p(V|I)$$,

where the $prior(I)$ encodes our prior knowledge about the sky (that it is non-negative, that it is sparse, or that we observed it before and know what it looked like). $p(V|I)$ is the likelihood function for independent Gaussian measurements where the means come from our model above and the noises come from the integration time. The independent Gaussian assumption is a really good one for visibilities since they come from averaging many voltage samples (more details to come for the expression).

It looks something like
$$ p(V|I) = \prod_i \frac{1}{\sigma_i \sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{(V_i - model(I_i))}{\sigma_i}\right)^2},$$

And we'll probably be optimizing the log posterior $log(p(I | V)$ since it's easier.

Ideally, we'd want to use MCMC to sample the full posterior distribution, but we are interested in the situations where we can't do that honestly (since the radio array might be large).

## Choice of prior
prior(I) encodes our belief about the sky before the observation and can be important for getting good solutions. A few we can play with:

None at all - we can just optimize the likelihood function $P(V|I)$. This will likely give ugly results since we're not giving it any information.

Gaussian - we can collect some statistics from some mocked observation of the same part of the sky (say, with a different spatial resolution, more noise, fewer sources, and things getting refracted by the ionosphere), and then use that as a prior. I guess this is similar to an L2 regularization on the image.

Laplace distribution - this corresponds to an L1 regularization on the image, which is what compressive sensing does in radio interferometry. I don't know if there's a well-motivated parameter to use for the distribution, though.